In [1]:
from utils.SystemMonitor import SystemMonitor
from utils.Table import Table
import time
import os

monitor = SystemMonitor()

Might want to drop notebook for sampling because of jupyter and kernel overhead

In [2]:
# ONLY RUN ONCE PER SAMPLE GROUP
file_name = "Samples1.csv"

if not os.path.exists("dataframes/" + file_name):
    Table.create_empty_df(file_name, ["TimeStamp", "Name", "CPU_CORES", "CPU_CORES_LOG", "CPU_FREQ", "RAM_TOTAL_GB", "RAM_AV_GB", "RAM_USED_GB", "SWAP_TOTAL_GB", "SWAP_USED_GB", "CPU%", "RAM%", "RC", "WC", "RB", "WB", "dt", "PRED"])
else:
    raise(FileExistsError)

FileExistsError: 

# **ResNet50**

In [3]:
from models.SS_ResNet50 import SS_ResNet50
ss_resnet50 = SS_ResNet50()

monitor.start(interval=0.1)
ss_resnet50.predict()
monitor.stop()

# monitor.compute_and_print_average()
# print("\nPrediction correct: " + str(ss_resnet50.correct_result()))

appendable = []
appendable.append(str(time.strftime("%y:%m:%d:%H:%M:%S", time.gmtime())))
appendable.append("ResNet50")

keys = monitor.compute_and_return_average()

for key in keys: # change to value not key
    appendable.append(keys[key])
    # appendable.append(key) # debug
appendable.append(str(ss_resnet50.correct_result()))

# if we change the model we can add model info
Table.append_to_df("Samples1.csv", [appendable])

# sanity check
print("Appended to samples file: \n")
print(appendable)


2023-10-27 08:22:58.937438: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 08:22:58.994638: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 08:22:58.996579: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-27 08:23:00.040043: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Monitoring started with an interval of 0.1 seconds.
1/1 [==============================] - 1s 1s/step
Monitoring stopped.


ValueError: Shape of passed values is (18, 1), indices imply (18, 18)